# Dog Race Classification Model

## Package Imports

In [2]:
from os import listdir

import pandas as pd

from PIL import Image

import tensorflow as tf

## Image Compression

In [8]:
for dataset in ["train", "test"]:
    original_directory = "./data/dogs/original/" + dataset + "/"
    compressed_directory = "./data/dogs/compressed/" + dataset + "/"

    original_files = listdir(original_directory)

    for file in original_files:
        image = Image.open(original_directory + file)
        ratio = image.size[0] / image.size[1]
        new_size = (round(64 * ratio), 64) if ratio > 1 else (64, round(64 / ratio))
        image.resize(new_size).save(compressed_directory + file)

## Label Loading

In [7]:
labels = pd.read_csv("./data/dogs/labels.csv")
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10222 non-null  object
 1   breed   10222 non-null  object
dtypes: object(2)
memory usage: 159.8+ KB
